In [3]:
from local_tree_decomp import *

from local_helpers import *
from helper_functions import *

from local_hom_count import *
from local_hom_count_int import *
from local_hom_count_int_pre import *
from local_hom_count_int_dict import *


In [4]:
def speed_test(graph, target_graph, naive=True):
    basic_result = count_homomorphisms(graph, target_graph)
    int_rep_result = count_homomorphisms_int(graph, target_graph)
    better_int_rep_result = count_homomorphisms_int_pre(graph, target_graph)
    dict_int_rep_result = count_homomorphisms_int_dict(graph, target_graph)

    results = [basic_result, int_rep_result, better_int_rep_result, dict_int_rep_result]

    if naive:
        brute_force_result = len(enumerate_homomorphisms(graph, target_graph))
        results.append(brute_force_result)

    if all(result == better_int_rep_result for result in results):
        print('result:', better_int_rep_result)

        if naive:
            print('\nBrute force:')
            %timeit len(enumerate_homomorphisms(graph, target_graph))
        
        print('\nBasic:')
        %timeit count_homomorphisms(graph, target_graph)
        
        print('\nInt representation:')
        %timeit count_homomorphisms_int(graph, target_graph)
    
        print('\nBetter int representation:')
        %timeit count_homomorphisms_int_pre(graph, target_graph)

        print('\n Dict int representation:')
        %timeit count_homomorphisms_int_dict(graph, target_graph)


In [5]:
graph = graphs.CompleteBipartiteGraph(1, 4)
target_graph = graphs.CompleteGraph(4)

speed_test(graph, target_graph)


result: 324

Brute force:
13.5 ms ± 223 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Basic:
9.19 ms ± 321 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Int representation:
5.55 ms ± 258 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Better int representation:
5.39 ms ± 303 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Better int representation:
5.32 ms ± 214 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
count_homomorphisms_int_dict(graph, target_graph)